In [3]:
!pip install pandas scikit-learn nltk spacy
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import pandas as pd

df = pd.read_csv("Resume.csv")  # update path if needed
df.head()



,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [3]:
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df = pd.read_csv("Resume.csv")

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)
job_description = """
Looking for a Python Developer with experience in Machine Learning,
Data Science, SQL, NLP, and REST APIs.
"""

job_clean = clean_text(job_description)


df['clean_resume'] = df['Resume_str'].apply(clean_text)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['clean_resume'])

job_vector = tfidf.transform([job_clean])

similarity_scores = cosine_similarity(job_vector, tfidf_matrix).flatten()
df['match_score'] = similarity_scores

ranked_df = df.sort_values(by='match_score', ascending=False)

ranked_df[['Resume_str', 'match_score']].head(5)
required_skills = ["python", "machine learning", "data science", "sql", "nlp", "api"]

def extract_skills(resume_text):
    found = []
    for skill in required_skills:
        if skill in resume_text.lower():
            found.append(skill)
    return found

def missing_skills(resume_text):
    found = extract_skills(resume_text)
    return list(set(required_skills) - set(found))

ranked_df['found_skills'] = ranked_df['Resume_str'].apply(extract_skills)
ranked_df['missing_skills'] = ranked_df['Resume_str'].apply(missing_skills)

ranked_df[['match_score', 'found_skills', 'missing_skills']].head(5)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,match_score,found_skills,missing_skills
1218,0.174516,"[python, machine learning, data science, sql, ...",[nlp]
1762,0.151824,"[python, machine learning, sql, api]","[nlp, data science]"
1142,0.133338,[sql],"[nlp, machine learning, api, python, data scie..."
926,0.131009,"[python, sql, api]","[nlp, machine learning, data science]"
1707,0.121159,"[machine learning, api]","[sql, nlp, python, data science]"


In [4]:
for i, row in ranked_df.head(3).iterrows():
    print("Candidate Score:", round(row['match_score'], 2))
    print("Skills Found:", row['found_skills'])
    print("Missing Skills:", row['missing_skills'])
    print("-" * 50)


Candidate Score: 0.17
Skills Found: ['python', 'machine learning', 'data science', 'sql', 'api']
Missing Skills: ['nlp']
--------------------------------------------------
Candidate Score: 0.15
Skills Found: ['python', 'machine learning', 'sql', 'api']
Missing Skills: ['nlp', 'data science']
--------------------------------------------------
Candidate Score: 0.13
Skills Found: ['sql']
Missing Skills: ['nlp', 'machine learning', 'api', 'python', 'data science']
--------------------------------------------------
